In [928]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [929]:
from bs4 import BeautifulSoup
import requests, re

In [930]:
df = pd.read_excel('data_files/감성사전_편집.xlsx')

In [931]:
df

,Unnamed: 0,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
0,가련하다,1,0,0,0,0,0,0,0,0,0,0
1,가슴 아프다,1,0,0,0,0,0,0,0,0,0,0
2,가슴앓이,1,0,0,0,0,0,0,0,0,0,0
3,가엾다,1,0,0,0,0,0,0,0,0,0,0
4,각박하다,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4005,광대,1,0,0,0,1,0,0,0,0,0,0
4006,한계였다,1,0,0,0,0,0,0,0,0,0,0
4007,멸망했다,1,0,0,0,0,0,0,0,0,0,0
4008,씁쓸한,1,0,0,0,0,0,0,0,0,0,0


In [932]:
df.set_index('Unnamed: 0', drop=True, inplace=True)

In [952]:
url = 'https://novel.naver.com/webnovel/detail?novelId=1051341&volumeNo=1'

In [953]:
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [954]:
text = soup.select('div.detail_view_content')[0].text
text = ' '.join(re.compile('[ㄱ-ㅎ가-힣]+').findall(text))

In [955]:
from konlpy.tag import Okt, Mecab
okt = Okt()
mecab = Mecab('C:/mecab/mecab-ko-dic/')

In [956]:
okt_li = okt.morphs(text)
mecab_li = mecab.morphs(text)

In [957]:
word_li = [i for i in okt_li if len(i) >= 2]

In [958]:
std_df = pd.read_csv('data_files/stopwords-ko.txt')

In [959]:
word_li = [i for i in word_li if i not in std_df ]

In [960]:
mood_count = {'슬픔': 0,
'분노': 0,
'기쁨': 0,
'긴장감': 0,
'무서운': 0,
'증오': 0,
'차분히': 0,
'섹시': 0,
'브릿락': 0,
'설렘': 0,
'몽환': 0}
mood_words = {}

for word in word_li:
    if word in df.index:
        word_mood = df.columns[df.loc[word, :] == 1].values[0]
        if word_mood in mood_words:
            mood_count[word_mood] += 1
            mood_words[word_mood].append(word)
        else:
            mood_count[word_mood] += 1
            mood_words[word_mood] =[word]
for mood in mood_words.keys():
    mood_words[mood] = set(mood_words[mood])

In [961]:
mood_count

{'슬픔': 20,
 '분노': 10,
 '기쁨': 15,
 '긴장감': 13,
 '무서운': 4,
 '증오': 6,
 '차분히': 3,
 '섹시': 5,
 '브릿락': 4,
 '설렘': 3,
 '몽환': 6}

In [962]:
mood_count.values()

dict_values([20, 10, 15, 13, 4, 6, 3, 5, 4, 3, 6])

In [943]:
mood_words

{'설렘': {'미소', '새로운', '즐거운', '즐기는'},
 '분노': {'강제'},
 '슬픔': {'고독', '멸망', '멸망했다', '씁쓸한', '연민', '홀로', '힘겹게'},
 '증오': {'비난', '새끼', '쓰레기'},
 '몽환': {'거짓', '시선', '의식', '환상'},
 '차분히': {'익숙한'},
 '섹시': {'가슴'},
 '긴장감': {'급히', '미쳐'},
 '기쁨': {'도움', '특별한', '효과'}}

In [891]:
df.loc['특별한', :]

슬픔     0
분노     0
기쁨     1
긴장감    0
무서운    0
증오     0
차분히    0
섹시     0
브릿락    0
설렘     0
몽환     0
Name: 특별한, dtype: int64

In [892]:
df

,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
단어,,,,,,,,,,,
가련하다,1,0,0,0,0,0,0,0,0,0,0
가슴 아프다,1,0,0,0,0,0,0,0,0,0,0
가슴앓이,1,0,0,0,0,0,0,0,0,0,0
가엾다,1,0,0,0,0,0,0,0,0,0,0
각박하다,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
야심,0,0,0,1,0,0,0,0,0,0,0
독종,0,0,0,1,1,0,0,0,0,0,0
기었고,1,0,0,0,0,0,0,0,0,0,0


In [893]:
text

'게임을 클리어하셨습니다 곧 엔딩이 시작됩니다 붉은 하늘 한가운데에 있는 거대한 검은 구멍을 바라보며 나는 중얼거렸다 끝났다 년 전 이 세계는 게임이 되었다 현실이 게임이 되어버렸다 우리들은 플레이어 가 되었고 게임 마스터 통칭 이라 불리는 자들에게 퀘스트를 받아 이야기를 진행했다 당연히 그것을 즐기는 건 우리가 아니다 저 하늘 위에 있는 아득한 존재 신 이라 불리는 이들을 위해 우리는 이 게임을 강제적으로 플레이해야 했다 그들은 수많은 이 운영하는 게임판 중 마음에 드는 장소에 참가하여 게임을 즐겼다 그저 지켜보는 이도 있었고 혹은 마음에 드는 플레이어를 자신의 아바타로 삼아 직접적으로 참여하는 자들도 있었다 그리고 그 결과 인류는 멸망했다 개쓰레기 같은 게임 같으니라고 나는 여태 수많은 게임을 플레이했지만 이것보다 쓰레기인 게임을 해본 적이 없었다 힘겹게 도달한 엔딩이 인류의 멸망이라니 적어도 마지막에라도 그 새끼들을 엿 먹이고 싶었는데 씁쓸한 미소가 입가에 맴돌았다 검은 구멍을 바라보았다 만약 내가 신격을 얻었다면 저 구멍에 들어갈 수 있겠지만 나는 누군가의 아바타가 된 몸이었다 당연히 누군가의 아바타가 되어서야 신격을 얻을 수 없었다 즉 나는 이 버려진 세계에서 홀로 살아가야 한다는 거지 녀석은 이런 나를 어떻게 생각할까 나를 아바타로 삼았던 신은 이런 나를 어떻게 생각할지 궁금했다 비웃을까 아니면 연민의 시선을 보낼까 아니 녀석이라면 이런 결과를 만든 나를 비난할지도 모른다 배드 엔딩 고독한 세계 를 달성하셨습니다 마침 들려온 건 녀석의 비난이 아닌 익숙한 시스템의 음성이었다 배드 엔딩이라니 하기야 지금 이 꼴이 배드 엔딩이 아니면 뭐겠는가 지금 생각하면 내가 이 게임을 플레이하며 조금만 다른 선택을 했더라면 이런 결말이 오지 않았을지도 모른다 허나 이미 도달한 결말을 바꿀 수는 없었다 홀로 모든 퀘스트를 클리어한 당신에게 경의를 표합니다 모든 플레이가 곧 종료됩니다 그동안 수고하셨습니다 그래 역시 이렇게 끝나는구나 나는 귓가에 아련히 들려오는 목소리

In [894]:
word_counts = {}
for i in word_li:
    if i in word_counts:
        word_counts[i] += 1
    else:
        word_counts[i] = 1

In [895]:
sorted(word_counts.items(), key = lambda item: item[1], reverse=True)

[('게임', 22),
 ('회차', 10),
 ('엔딩', 8),
 ('있는', 6),
 ('플레이', 6),
 ('없었다', 6),
 ('상점', 6),
 ('이다', 6),
 ('포인트', 6),
 ('구매', 6),
 ('클리어', 5),
 ('시작', 5),
 ('됩니다', 5),
 ('으로', 5),
 ('이런', 5),
 ('나를', 5),
 ('생각', 5),
 ('설명', 5),
 ('하셨습니다', 4),
 ('세계', 4),
 ('했다', 4),
 ('하는', 4),
 ('아바타', 4),
 ('인류', 4),
 ('에서', 4),
 ('패키지', 4),
 ('설마', 4),
 ('구멍', 3),
 ('우리', 3),
 ('플레이어', 3),
 ('있었다', 3),
 ('쓰레기', 3),
 ('같은', 3),
 ('도달', 3),
 ('녀석', 3),
 ('배드', 3),
 ('지금', 3),
 ('모든', 3),
 ('하나', 3),
 ('싱글', 3),
 ('있다', 3),
 ('현재', 3),
 ('부터', 3),
 ('다시', 3),
 ('특전', 3),
 ('까지', 3),
 ('하늘', 2),
 ('검은', 2),
 ('되었다', 2),
 ('이라', 2),
 ('불리는', 2),
 ('에게', 2),
 ('퀘스트', 2),
 ('받아', 2),
 ('이야기', 2),
 ('당연히', 2),
 ('그것', 2),
 ('수많은', 2),
 ('마음', 2),
 ('드는', 2),
 ('하여', 2),
 ('혹은', 2),
 ('자신', 2),
 ('결과', 2),
 ('여태', 2),
 ('멸망', 2),
 ('이라니', 2),
 ('만약', 2),
 ('신격', 2),
 ('누군가', 2),
 ('이었다', 2),
 ('홀로', 2),
 ('한다는', 2),
 ('거지', 2),
 ('어떻게', 2),
 ('아니면', 2),
 ('아니', 2),
 ('라면', 2),
 ('비난', 2),
 ('모른다', 2),
 ('아닌', 2),

In [909]:
'급히' in df.index

False

In [908]:
df.columns[df.loc['고독'] == 1]

Index(['슬픔'], dtype='object')

In [470]:
mood_dic = {'슬픔': 0,
'분노': 1,
'기쁨': 2,
'긴장감': 3,
'무서운': 4,
'증오': 5,
'차분히': 6,
'섹시': 7,
'브릿락': 8,
'설렘': 9,
'몽환': 10}

In [471]:
def append_word(word, mood_li):
    columns_num = [0,0,0,0,0,0,0,0,0,0,0]
    for mood in mood_li:
        num = mood_dic[mood]
        columns_num[num] += 1
    columns_num = tuple(columns_num)
    tem_df = pd.DataFrame(data=[columns_num], index=[word], columns=['슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', '차분히', '섹시', '브릿락', '설렘', '몽환'])
    if word not in df.index:
        return df.append(tem_df)
    else:
        return df

In [910]:
df = append_word('급히', ['긴장감'])

In [911]:
df.tail(3)

,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
멸망했다,1,0,0,0,0,0,0,0,0,0,0
씁쓸한,1,0,0,0,0,0,0,0,0,0,0
급히,0,0,0,1,0,0,0,0,0,0,0


In [912]:
df.to_excel('data_files/감성사전_편집.xlsx')

In [810]:
df.drop('보람', inplace=True)